In [5]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots

In [6]:
ls = Vector{Any}(undef, 13) # there are 13 columns
ls[1] = String31 
for i in 2:13
    ls[i] = Float64
end

In [7]:
#load the data and orgnize 
cap_cstr = CSV.read("/home/xue.326/julia_env/LiBattery_Env/data/capacity_constraints.csv", 
                    DataFrame, 
                    header=1,
                    types=ls) |> DataFrame

distance = CSV.File("/home/xue.326/julia_env/LiBattery_Env/data/distance.csv",header=1) |> DataFrame

LCA_model = CSV.File("/home/xue.326/julia_env/LiBattery_Env/data/LCA_model314.csv",header=1) |> DataFrame

SD = CSV.File("/home/xue.326/julia_env/LiBattery_Env/data/emission_sink.csv",header=1) |> DataFrame; 

# first(cap_cstr, 5)
# first(distance, 5)
# ncol(distance)

In [8]:
first(cap_cstr, 5)

Row,Country,Li,Co,Mn,Ni,NMC111 powder,Graphite,PP,PE,Cu,Al,electricity,heat
,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,10800.0,0.0,0.0,0.0,0.0,0.0,300000.0,0.0,0.0,0.0,0.0,0.0
2,Australia,37000.0,7400.0,3.0e6,189000.0,0.0,500.0,500000.0,600000.0,920000.0,1.6e6,0.0,0.0
3,Bahrain,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2e6,0.0,0.0
4,Bolivia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Brazil,1800.0,0.0,1.2e6,135000.0,0.0,87000.0,1.9e6,3.6e6,400000.0,590000.0,0.0,0.0


In [9]:
LCA_model

Row,process,input (material/kg battery),input (material/kWh battery),EF (kg CO2/material),input (material/kg NMC111 powder)
,String15,Float64,Float64,Float64,Float64
1,Li,0.00905,0.0635,0.0155,0.072
2,Co,0.051,0.358,0.492,0.203
3,Mn,0.0479,0.336,5.6,0.19
4,Ni,0.051,0.358,1.07,0.203
5,NMC111 powder,0.252,1.77,14.7,0.0
6,Graphite,0.141,0.986,4.86,0.0
7,PP,0.011,0.0774,2.53,0.0
8,PE,0.00364,0.0255,2.93,0.0
9,Cu,0.117,0.824,3.08,0.0


In [10]:
w_cell = 164.98                             # kg/pack
cap_cell = 23.5                             # kWh/pack
cell_demand = 6000000                       # annual demand of Li-ion battery for tesla (2M EV/yr, 3 NMC111 pack/EV)
gobal_sink = 1.53e9                         # global CO2 sequestration
global_emission = 4.75e10                   # global CO2 emission
EF_aircraft = 0.433                         # kg CO2/km*ton emission factor for freight transporation 
EF_input = LCA_model[!,"EF (kg CO2/material)"]
process = LCA_model[!,"process"]
countries = cap_cstr[!,"Country"]
n = size(countries,1)                       # No. of countries
m = size(process,1);                        # No. of processes 

In [11]:
# seperate model
ups_powder = [1, 2, 3, 4]      # upstream processes for NMC111 powder
ups_cell = [i for i in 5:10];  # upstream processes for Li-ion battery manufacturing

scaler_powder = LCA_model[1:4,"input (material/kg NMC111 powder)"]     # mineral inputs for 1 kg NMC111 powder
scaler_cell = LCA_model[!, "input (material/kg battery)"];             # inputs for 1 kg NMC111 cell

In [12]:
FU_mass = cell_demand * w_cell      # input amount per FU
input_FU = DataFrame(process = LCA_model[!,"process"], inputs = FU_mass .* LCA_model[!,"input (material/kg battery)"]);
input_FU = input_FU[!, "inputs"]
# first(input_FU, 5)

11-element Vector{Float64}:
 8.958414e6
 5.048387999999999e7
 4.741525199999999e7
 5.048387999999999e7
 2.4944975999999997e8
 1.3957307999999997e8
 1.0888679999999998e7
 3.6031631999999997e6
 1.1581596e8
 2.3658131999999997e8
 1.1710280399999999e8

----

In [27]:
model = Model(Gurobi.Optimizer);

Academic license - for non-commercial use only - expires 2023-11-24


In [33]:
#variables 
@variable(model, x[1:n, 1:m] >= 0)         # x[i,k] production amount of product k at location i
@variable(model, y[1:n, 1:n, 1:m] >= 0);   # y[i,j,k] ship product k from i to j 


# node output flow constraint
for k in 1:m
    for i in 1:n
        @constraint(model, sum(y[i,j,k] for j in 1:n) <= x[i,k])
    end
end


# NMC111 powder input flow constraint
for k in ups_powder
    for j in 1:n
        @constraint(model, sum(y[i,j,k] for i in 1:n) >= x[j,5] * scaler_powder[k])
    end
end


# Li-ion battery input flow constraint
for k in ups_cell
    for j in 1:n
        @constraint(model, sum(y[i,j,k] for i in 1:n) >= x[j,11] * scaler_cell[k])
    end
end


# production capacity constraints
for i in 1:n
    for k in 1:m
        @constraint(model, x[i,k] <= cap_cstr[!, 2:end][i,k])
    end
end


# final demand constraints
for k in 1:m
    @constraint(model, sum(x[i,k] for i in 1:n) >= input_FU[k])
end

In [34]:
proc_emi = []

for k in 1:m
    emi = sum(x[i, k] for i in 1:n)
    push!(proc_emi, emi)
end

tot_pemi = EF_input' * proc_emi;

In [35]:
trans_emi = 0

for k in 1:m
    for i in 1:n
        for j in 1:n
            trans_emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_aircraft
        end
    end
end

cell_output = sum(x[i,11] for i in 1:n)
tot_temi = trans_emi + sum(cell_output * distance[!, 2:end][i,38] for i in 1:n);

In [36]:
# supply - based on emission
tot_sink = 0
for k in 1:m
    for i in 1:n
        emi = x[i,k] * EF_input[k]
        S_country = SD[!, "sink ton/yr"][i] * emi / SD[!, "emission"][i]
        S_global = gobal_sink * emi / global_emission
        S = S_country + S_global
        tot_sink += S
    end
end

In [37]:
@objective(model, Max, tot_sink - tot_temi);

In [38]:
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1291 rows, 18040 columns and 35320 nonzeros
Model fingerprint: 0xc300a0fd
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [5e-04, 4e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+15]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve time: 0.01s

Solved with dual simplex
Solved in 0 iterations and 0.01 seconds
Infeasible or unbounded model

User-callback calls 43, time in user-callback 0.00 sec


In [20]:
first(SD,5)

Row,country,emission,sink ton/yr
,String31,Float64,Float64
1,Argentina,1.98e8,1.79e8
2,Australia,3.89e8,2.87e8
3,Bahrain,3.33e7,72500.0
4,Bolivia,8.97e7,1.73e8
5,Brazil,8.19e8,1.81e9
